# Automatizar la descarga de información de descarga de la API

In [2]:
# Cargamos las librerias
import pandas as pd
import requests

In [3]:
# Lo agregamos a nuestro código para que no se detenga la ejecución del programa
import time

payload = {'api_key': 'sYrr0KtFovL8YGwcaljKqnYzCVJal8XZ3Q0rled6'}
endpoint = 'https://api.nasa.gov/neo/rest/v1/neo/browse'

dict_datos = {}

for i in range(0, 10):
    try:
        time.sleep(5)   # Para no saturar de peticiones el servidor
        r = requests.get(endpoint, params= payload, timeout= 5)     # Se usa el timeout para que si no hay respuesta continue, seria un error de petición

        if r.status_code == 200:
            json = r.json()

            # Cargamos los datos de la key seleccionada en nuestro diccionario
            data = json['near_earth_objects']
            dict_datos[i] = data

            # Actualizamos la dirección de endpoint para el proximo pedido de información
            new_link = json['links']['next']
            endpoint = new_link

    except:
        continue

#### Concatenación

In [4]:
# Vamos a reemplazar los datos en formato diccionario por el contenido de un dataframe 
for elemento in dict_datos: # Lee la primera clave en este caso los valores que generamos con range(0, 10)
    normalized = pd.json_normalize(dict_datos[elemento])
    df = pd.DataFrame.from_dict(normalized) # Se puede omitir esta linea de codigo
    dict_datos[elemento] = df

In [5]:
# Cada key contiene un dataframe
primer_df = dict_datos[0]

primer_df.head()

,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range
0,2000433,2000433,433 Eros (A898 PA),Eros,433,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,10.41,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,...,1.133430877277801,178.8953167089443,1.782924415693006,2460445.650244578800,334.7274812953938,.5597426756016515,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
1,2000719,2000719,719 Albert (A911 TB),Albert,719,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,15.59,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,...,1.194434733405151,156.2221413583312,4.077803107571985,2459955.944856209393,102.3722423709411,.2302801886354059,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,13.88,False,"[{'close_approach_date': '1910-01-04', 'close_...",False,...,1.060514970150709,350.478081294877,3.883763462379203,2460678.627539692991,289.4757166026648,.2535681417064373,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
3,2001036,2001036,1036 Ganymed (A924 UB),Ganymed,1036,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,9.25,False,"[{'close_approach_date': '1910-02-25', 'close_...",False,...,1.245193977307087,132.4833028411867,4.08579192023869,2460569.648416864637,321.6905687592489,.2264841253075913,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
4,2001221,2001221,1221 Amor (1932 EA1),Amor,1221,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,17.38,False,"[{'close_approach_date': '1900-03-08', 'close_...",False,...,1.084248654678618,26.67573388569273,2.755201145123765,2460838.656012985569,197.6417532004091,.370548941445061,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU


#### Ejemplos de uso de pd.concat()

In [6]:
# Uso de concat en Series

# Creamos 2 series para los ejemplos
serie_1 = pd.Series([1, 2, 3], index= ['a', 'b', 'c'])
serie_2 = pd.Series([4, 5, 6], index= ['d', 'e', 'f'])

print('serie_1:')
print(serie_1)
print('\nserie_2')
print(serie_2)

serie_1:
a    1
b    2
c    3
dtype: int64

serie_2
d    4
e    5
f    6
dtype: int64


In [7]:
# Union de las series a lo largo de las filas
pd.concat([serie_1, serie_2], axis= 0)

a    1
b    2
c    3
d    4
e    5
f    6
dtype: int64

In [8]:
# Union de las series a lo largo de las columnas
pd.concat([serie_1, serie_2], axis= 1)


,0,1
a,1.0,NaN
b,2.0,NaN
c,3.0,NaN
d,NaN,4.0
e,NaN,5.0
f,NaN,6.0


In [9]:
# En el caso anterior no coinciden los indices por eso se agrupa en 2 columnas generando NaN en los lugares sin info
# Creamos una tercera serie
serie_3 = pd.Series([7, 8, 9], index= ['a', 'b', 'c'])

# Concateno (uno las series) por columnas y agrego key para los nombres de las columnas
pd.concat([serie_1, serie_3], axis= 1, keys= ['serie_1', 'serie_3'])

,serie_1,serie_3
a,1,7
b,2,8
c,3,9


In [10]:
# Veamos que pasa si concatenamos estas series con iguales indices
pd.concat([serie_1, serie_3], axis=0)

a    1
b    2
c    3
a    7
b    8
c    9
dtype: int64

In [11]:
# Para solucionar el ejemplo anterior podemos generar un nuevo indice borrando el que traen las series
pd.concat([serie_1, serie_3], axis= 0).reset_index(drop= True)

0    1
1    2
2    3
3    7
4    8
5    9
dtype: int64

In [12]:
# Si necesitamos mantener los indices y queremos identificarlos, podemos agregar otro nivel a nuestro indice
pd.concat([serie_1, serie_3], axis= 0, keys= ['serie_1', 'serie_3'])

serie_1  a    1
         b    2
         c    3
serie_3  a    7
         b    8
         c    9
dtype: int64

In [13]:
# Acceder a elementos cuando tenemos multiindices
concat_1 = pd.concat([serie_1, serie_3], axis= 0, keys= ['serie_1', 'serie_3'])

concat_1.loc[('serie_1', 'b')]


2

In [14]:
# Otro elemento
concat_1.loc[('serie_3', 'b')]

8

In [15]:
# Ahora haremos lo mismo pero con multiples Series
serie_4 = pd.Series([10, 11, 12], index=['a', 'b', 'c'])
serie_5 = pd.Series([13, 14, 15], index=['a', 'b', 'c'])

concat_2 = pd.concat([serie_1, serie_3, serie_4, serie_5], axis=0, keys=['serie_1', 'serie_3', 'serie_4', 'serie_5'])

concat_2

serie_1  a     1
         b     2
         c     3
serie_3  a     7
         b     8
         c     9
serie_4  a    10
         b    11
         c    12
serie_5  a    13
         b    14
         c    15
dtype: int64

In [16]:
# Accedemos a una de las series por el indice principal
concat_2.loc['serie_4']

a    10
b    11
c    12
dtype: int64

#### concat con DataFrame


In [17]:
# Creamos un diccionario para los ejemplos
data_1 = {'column_1': [1, 2, 3],
          'column_2': [4, 5, 6]
          }

# Creamos un DataFrame con nuestro diccionario y asignamos un indice
df_1 = pd.DataFrame(data_1, index=['a', 'b', 'c'])

df_1

,column_1,column_2
a,1,4
b,2,5
c,3,6


In [18]:
# Realizamos la misma operación para un segundo DataFrame
data_1 = {'column_1': [7, 8, 9],
          'column_2': [10, 11, 12]
          }

df_2 = pd.DataFrame(data_1, index=['d', 'e', 'f'])

df_2

,column_1,column_2
d,7,10
e,8,11
f,9,12


In [19]:
# Concatenamos (unimos), por filas 
pd.concat([df_1, df_2], axis=0)

,column_1,column_2
a,1,4
b,2,5
c,3,6
d,7,10
e,8,11
f,9,12


In [20]:
# Concatenamos (unimos), por columnas 
pd.concat([df_1, df_2], axis=1)

,column_1,column_2,column_1,column_2
a,1.0,4.0,NaN,NaN
b,2.0,5.0,NaN,NaN
c,3.0,6.0,NaN,NaN
d,NaN,NaN,7.0,10.0
e,NaN,NaN,8.0,11.0
f,NaN,NaN,9.0,12.0


In [21]:
# Creamos un tercer DataFrame con el mismo indice del primero
data_3 = {'column_3': [7, 8, 9],
          'column_4': [10, 11, 12]
          }

df_3 = pd.DataFrame(data_3, index=['a', 'b', 'c'])

df_3

,column_3,column_4
a,7,10
b,8,11
c,9,12


In [22]:
# Realizamos la misma operacion, esta vez comparten los mismos indices
# Concatenamos por columnas
pd.concat([df_1, df_3], axis=1)

,column_1,column_2,column_3,column_4
a,1,4,7,10
b,2,5,8,11
c,3,6,9,12


In [23]:
# Creamos un cuarto DataFrame con el mismo indice del primero
data_4 = {'column_1': [7, 8, 9],
          'column_2': [10, 11, 12]
          }

df_4 = pd.DataFrame(data_4, index=['a', 'b', 'c'])

df_4

,column_1,column_2
a,7,10
b,8,11
c,9,12


In [24]:
# Concatenamos por filas, con iguales indices, asignando un nuevo nivel de indices con keys
df_concat = pd.concat([df_1, df_4], axis=0, keys=['df_1', 'df_4'])

df_concat

column_1  column_2
df_1 a         1         4
     b         2         5
     c         3         6
df_4 a         7        10
     b         8        11
     c         9        12

In [25]:
# Accedemos al dataframe de indice df_1
df_concat.loc['df_1']

,column_1,column_2
a,1,4
b,2,5
c,3,6


In [26]:
# Ahora accedemos a la fila 'b' dentro de df_1
df_concat.loc[('df_1', 'b')]

column_1    2
column_2    5
Name: (df_1, b), dtype: int64

#### Union de Dataframes

In [27]:
# Para unir cada df vamos a crear una lista de df
# Primero creamos una lista vacía
lista_de_dataframes = []

# Con un for vamos a recorrer los indices para asignar cada df como elemento de la lista
for key in dict_datos:
    lista_de_dataframes.append(dict_datos[key])

# Ahora vamos a pasar el listado de df a concat, y vamos a unir por filas,reiniciando el indice
df_completo = pd.concat(lista_de_dataframes, axis= 0).reset_index(drop= True)

In [28]:
# Nuestro df completo
print(f'El DataFrame completo tiene {df_completo.shape[0]} filas y {df_completo.shape[1]} columnas')
df_completo.head(7)

El DataFrame completo tiene 200 filas y 44 columnas


,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range
0,2000433,2000433,433 Eros (A898 PA),Eros,433,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,10.41,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,...,1.133430877277801,178.8953167089443,1.782924415693006,2460445.650244578800,334.7274812953938,.5597426756016515,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
1,2000719,2000719,719 Albert (A911 TB),Albert,719,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,15.59,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,...,1.194434733405151,156.2221413583312,4.077803107571985,2459955.944856209393,102.3722423709411,.2302801886354059,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,13.88,False,"[{'close_approach_date': '1910-01-04', 'close_...",False,...,1.060514970150709,350.478081294877,3.883763462379203,2460678.627539692991,289.4757166026648,.2535681417064373,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
3,2001036,2001036,1036 Ganymed (A924 UB),Ganymed,1036,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,9.25,False,"[{'close_approach_date': '1910-02-25', 'close_...",False,...,1.245193977307087,132.4833028411867,4.08579192023869,2460569.648416864637,321.6905687592489,.2264841253075913,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
4,2001221,2001221,1221 Amor (1932 EA1),Amor,1221,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,17.38,False,"[{'close_approach_date': '1900-03-08', 'close_...",False,...,1.084248654678618,26.67573388569273,2.755201145123765,2460838.656012985569,197.6417532004091,.370548941445061,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
5,2001566,2001566,1566 Icarus (1949 MA),Icarus,1566,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,16.57,True,"[{'close_approach_date': '1902-06-11', 'close_...",False,...,.1865939396616765,31.43692555751435,1.969582506453463,2460417.813945689359,344.755307383562,.8804863368496817,J2000,APO,Near-Earth asteroid orbits which cross the Ear...,a (semi-major axis) > 1.0 AU; q (perihelion) <...
6,2001580,2001580,1580 Betulia (1950 KA),Betulia,1580,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,14.69,False,"[{'close_approach_date': '1911-05-07', 'close_...",False,...,1.125685910234569,159.6732940100622,3.266587314844516,2460735.655395124742,258.5011176005039,.3028412607283829,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
